In [ ]:
# 📘 FX Reinforcement Learning System（簡易版）
"""
- 作成者：fxinsightslab
- バージョン：v1.0
- 公開日：2025年4月xx日
- ライセンス：個人利用限定（再配布・商用利用・改変配布禁止）

---

## 📄 利用に関する注意事項

- このノートブックは学習・研究・検証目的で提供されています。
- 実際の売買判断やトレード運用は自己責任の下で行ってください。
- 本テンプレートを改変しての再配布、または第三者への提供・販売は禁止です。

> 💬 ご質問や不具合の報告は、X（旧Twitter）[@fxinsightslab](https://x.com/fxinsightslab) までお気軽にどうぞ！

---

✅ 以下のセルから順次実行すると最後の評価まで行えるようになっています。
"""

In [9]:
# データ準備
################################################################
# 指定されたプロジェクトルートからノートブックを実行し、
# MT5からGOLDのM5データを取得・前処理して表示する
################################################################
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from sklearn.preprocessing import StandardScaler
import os


# --- プロジェクトルートを自動検出する関数 ---
def find_project_root(target_folder_name="FX Reinforcement Learning System（簡易版）"):
    current_path = os.getcwd() # 現在の作業ディレクトリを取得
    while True:
        # 現在のフォルダ名がターゲット名と一致するか確認
        if os.path.basename(current_path) == target_folder_name:
            return current_path # 一致すればそのパスを返す
        parent = os.path.dirname(current_path)# 一つ上の階層へ移動
        if parent == current_path:
            # ルートまで探索しても見つからなければエラーを投げる
            raise FileNotFoundError(f"{target_folder_name} フォルダが上位ディレクトリに見つかりませんでした。")
        current_path = parent# 次のループでは上位フォルダを探索

try:# プロジェクトルートのパスを取得し、指定ノートブックを実行
    base_path = find_project_root()  
    target_path = os.path.join(base_path, "src/utils/data_preparation.ipynb").replace("\\", "/")
    %run "$target_path"# ノートブックを実行して関数などを読み込み
except FileNotFoundError as e:
    print(e)# 見つからなければエラーメッセージを表示


# --- MT5からデータ取得 ---
mt5.initialize()  # MT5との接続を初期化
symbol = "GOLD"  # 取得する銘柄を指定
# GOLDの5分足データを「現在の最新の足」から遡って1000本取得
rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 10000)  # 0 は「現在時刻に最も近いバー（=最新バー）を起点」という意味
mt5.shutdown()  # MT5との接続を終了

# --- データフレームに変換・整形 ---
df = pd.DataFrame(rates)  # MT5データをDataFrame形式に変換
df["time"] = pd.to_datetime(df["time"], unit="s")  # UNIX時間を日時形式に変換

# --- 前処理実行（data_preparation.ipynbで定義された関数） ---
df = prepare_data(df)  # 指定関数でテクニカル指標などを追加・整形

# --- 結果表示（先頭5行） ---
print(df.head())  # データの先頭を表示して確認

                  time      open      high       low     close  tick_volume  \
49 2025-03-06 10:40:00 -1.352176 -1.354161 -1.352327 -1.357108          590   
50 2025-03-06 10:45:00 -1.356912 -1.359425 -1.365901 -1.370913          704   
51 2025-03-06 10:50:00 -1.370243 -1.363204 -1.373502 -1.359071         1537   
52 2025-03-06 10:55:00 -1.358740 -1.358210 -1.357553 -1.355552          925   
53 2025-03-06 11:00:00 -1.355356 -1.359897 -1.383547 -1.383499         1161   

    spread  real_volume    volume  raw_close    SMA_20    EMA_50    RSI_14  \
49      27            0 -0.384013    2904.39 -1.304016 -1.273866 -2.074167   
50      27            0 -0.125270    2902.35 -1.310224 -1.277522 -2.319547   
51      27            0  1.765370    2904.10 -1.316099 -1.280569 -1.746484   
52      27            0  0.376328    2904.62 -1.321153 -1.283358 -1.584495   
53      27            0  0.911972    2900.49 -1.326887 -1.287135 -2.121201   

        MACD    ATR_14  BB_upper  BB_lower       OBV  zi

In [10]:
# ========== ボラティリティ分析AI用の強化学習環境 ==========
################################################################
# 状態空間	TrendEnvと同様の13指標（ボラティリティ系も含む）
# 行動空間	0=保留、1=リスク回避、2=エントリー促進
# 報酬設計	ATRによるペナルティ（リスク回避）／価格変動による報酬（エントリー促進）
# 拡張性	価格変化のしきい値、標準偏差の導入、報酬感度などを調整可能
################################################################
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class VolatilityEnv(gym.Env):
    def __init__(self, data):
        super(VolatilityEnv, self).__init__()
        self.data = data.reset_index(drop=True)  # インデックスをリセット
        self.current_step = 0  # ステップカウンタ初期化
        self.max_steps = len(data) - 1  # 最大ステップ数をデータ長で定義

        # --- 状態空間：13特徴量（TrendEnvと統一）---
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(13,), dtype=np.float32
        )

        # --- 行動空間 ---
        # 0: 保留、1: リスク回避、2: エントリー促進
        self.action_space = spaces.Discrete(3)

    # --- 現在の状態を返す（13次元ベクトル）---
    def _get_state(self):
        row = self.data.iloc[self.current_step]
        return np.array([
            row["close"],
            row["SMA_20"],
            row["RSI_14"],
            row["MACD"],
            row["ATR_14"],
            row["BB_upper"],
            row["BB_lower"],
            row.get("volume", 0.0),  # volume がない場合は 0 にする
            row["open"],
            row["high"],
            row["low"],
            row["EMA_50"],
            row["OBV"]
        ], dtype=np.float32)

    # --- 環境のリセット ---
    def reset(self, seed=None):
        self.current_step = 0  # 最初のステップに戻る
        return self._get_state(), {}

    # --- 1ステップ進める処理 ---
    def step(self, action):
        reward = 0  # 初期報酬
        terminated = False
        truncated = False

        # --- リスク回避行動（ボラティリティが高いときに有効）---
        if action == 1:
            # ATR（平均的な価格変動幅）に基づいてペナルティ（=市場不安定）
            reward = -self.data["ATR_14"].iloc[self.current_step] * 0.2

        # --- エントリー促進行動（価格が大きく動いたときに有効）---
        elif action == 2:
            price_diff = (
                self.data["close"].iloc[self.current_step]
                - self.data["close"].iloc[self.current_step - 1]
            )
            reward = price_diff * 0.5  # 感度を調整（0.5倍）

        # --- ステップ更新 ---
        self.current_step += 1
        if self.current_step >= self.max_steps:
            terminated = True  # データ末尾に達したら終了

        # --- 次の状態と報酬を返す ---
        next_state = self._get_state()
        return next_state, reward, terminated, truncated, {}


In [13]:
# ========== ② 環境＆学習設定 ==========
################################################################
# load_or_create_model() により学習済みモデルの再利用 or 上書きが簡単に切り替え可能。
################################################################

import os
from stable_baselines3 import PPO


# --- モデルの読み込み or 新規作成を行う関数 ---
def load_or_create_model(env, model_name="volatility_ai_ppo", force_new=False, verbose=1):
    """
    モデルの読み込み or 新規作成

    Parameters:
        env: 強化学習環境（Monitor付き）
        model_name: モデルの保存ファイル名（拡張子なし）
        force_new: Trueなら既存モデルを無視して新規作成
        verbose: PPOの出力詳細レベル

    Returns:
        PPOモデルインスタンス
    """
    base_path = find_project_root()  # プロジェクトルートパスを取得
    model_path = os.path.join(base_path, f"src/agents/models/{model_name}.zip")  # モデルファイルパスを生成

    # 既存モデルがあり、再学習指示でなければロード
    if os.path.exists(model_path) and not force_new:
        print(f"✅ 既存モデル '{model_name}' を読み込みます。")
        model = PPO.load(model_path)
        model.set_env(env)  # 環境を再設定（再学習時に必要）
    else:
        # 既存ファイルがあっても force_new = True の場合は上書き
        if os.path.exists(model_path):
            print(f"🆕 モデル '{model_name}' を上書きして新規学習します。")
        else:
            print(f"🆕 モデル '{model_name}' が存在しないため、新規に作成します。")
        model = PPO("MlpPolicy", env, verbose=verbose)  # 新規モデル作成

    return model

# --- 必要な補助ツールのインポート ---
from stable_baselines3.common.monitor import Monitor  # ログ記録用ラッパー
from stable_baselines3.common.env_checker import check_env  # 環境構成の妥当性チェック

# --- 環境の初期化 ---
env = VolatilityEnv(df)       # ボラティリティ分析用の環境を構築
check_env(env)              # Gym互換チェック（デバッグ用）
monitored_env = Monitor(env)  # モニター付き環境で学習ログ等を記録

# --- 学習設定（force_train=Trueで新規学習を強制）---
force_train = False  # ← Trueなら新規に学習を始める、Falseなら保存済みモデルを再利用

# --- モデル取得（ロード or 新規作成）---
model = load_or_create_model(monitored_env, model_name="volatility_ai_ppo", force_new=force_train)

# --- モデルの学習実行 ---
model.learn(total_timesteps=100000)  # 総学習ステップ数を指定（例：10000）

# --- モデルの保存（自動で .zip 拡張子が付加される）---
model.save("models/volatility_ai_ppo")
print("✅ 学習完了・モデル保存済み")

✅ 既存モデル 'volatility_ai_ppo' を読み込みます。
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 853  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 445         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.002321069 |
|    clip_fraction        | 0.0213      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.465      |
|    explained_variance   | 0.765       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.387       |
|    n_updates            | 650         |
|    policy_gradient_loss | -0.000839   |
|    value_loss           | 0.857       |
----

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.95e+03    |
|    ep_rew_mean          | 709         |
| time/                   |             |
|    fps                  | 279         |
|    iterations           | 12          |
|    time_elapsed         | 87          |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.002481816 |
|    clip_fraction        | 0.0319      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.338      |
|    explained_variance   | 0.745       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.534       |
|    n_updates            | 750         |
|    policy_gradient_loss | -0.0034     |
|    value_loss           | 0.934       |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 9.95

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 9.95e+03     |
|    ep_rew_mean          | 712          |
| time/                   |              |
|    fps                  | 281          |
|    iterations           | 22           |
|    time_elapsed         | 160          |
|    total_timesteps      | 45056        |
| train/                  |              |
|    approx_kl            | 0.0022174153 |
|    clip_fraction        | 0.0175       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.223       |
|    explained_variance   | 0.758        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.502        |
|    n_updates            | 850          |
|    policy_gradient_loss | -0.00132     |
|    value_loss           | 1.28         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 9.95e+03     |
|    ep_rew_mean          | 717          |
| time/                   |              |
|    fps                  | 280          |
|    iterations           | 32           |
|    time_elapsed         | 233          |
|    total_timesteps      | 65536        |
| train/                  |              |
|    approx_kl            | 0.0008531823 |
|    clip_fraction        | 0.00962      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.187       |
|    explained_variance   | 0.8          |
|    learning_rate        | 0.0003       |
|    loss                 | 0.288        |
|    n_updates            | 950          |
|    policy_gradient_loss | -0.00158     |
|    value_loss           | 1.08         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 9.95e+03     |
|    ep_rew_mean          | 717          |
| time/                   |              |
|    fps                  | 273          |
|    iterations           | 42           |
|    time_elapsed         | 314          |
|    total_timesteps      | 86016        |
| train/                  |              |
|    approx_kl            | 0.0016537777 |
|    clip_fraction        | 0.017        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.106       |
|    explained_variance   | 0.777        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.507        |
|    n_updates            | 1050         |
|    policy_gradient_loss | -0.00206     |
|    value_loss           | 1.26         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

In [5]:
import os
from stable_baselines3 import PPO
import zipfile

################################################################
# os.path.exists()	モデルファイルの存在確認（trend_ai_ppo.zip）
# PPO.load()	学習済みモデルの読み込み（stable-baselines3）
# zipfile.ZipFile	.zip の中に含まれるファイルを列挙表示
################################################################

# --- モデルの読み込み処理(find_project_rootを使用) ---
try:
    base_path = find_project_root()  # プロジェクトルートを自動検出
    model_path = os.path.join(base_path, "src/agents/models/volatility_ai_ppo.zip")  # モデルのファイルパスを指定

    if os.path.exists(model_path):
        print("✅ モデルファイルが見つかりました:", model_path)

        try:
            # --- モデルの読み込みを試みる ---
            model = PPO.load(model_path)
            print("✅ モデルの読み込みに成功しました！")

            # --- zipファイルの内容確認（中身をリスト表示）---
            with zipfile.ZipFile(model_path, 'r') as zip_ref:
                print("📦 含まれるファイル一覧:")
                for name in zip_ref.namelist():
                    print("  -", name)

        except Exception as e:
            # モデルファイルが壊れている・依存関係が合わない等の可能性
            print("❌ モデルの読み込みに失敗しました:", e)
    else:
        print("❌ モデルファイルが存在しません:", model_path)

except FileNotFoundError as e:
    # プロジェクトルートが見つからなかった場合のエラー処理
    print("❌ エラー:", e)

✅ モデルファイルが見つかりました: C:\Users\kotak\FX Reinforcement Learning System（簡易版）\src/agents/models/volatility_ai_ppo.zip
✅ モデルの読み込みに成功しました！
📦 含まれるファイル一覧:
  - data
  - pytorch_variables.pth
  - policy.pth
  - policy.optimizer.pth
  - _stable_baselines3_version
  - system_info.txt


In [14]:
# ========== PnLベースのモデル評価関数（ロング＆ショート両対応） ==========

def evaluate_model_with_pnl(model, env, steps=500):
    obs, _ = env.reset()  # 環境初期化
    df = env.data.reset_index(drop=True)  # データフレームをリセット

    position = None  # 現在のポジション（None / "long" / "short"）
    entry_price = 0  # エントリー価格
    entry_step = 0  # エントリーステップ番号
    capital = 10000  # 評価開始時の仮想資金
    equity_curve = [capital]  # 資産の推移を記録
    trade_log = []  # 各トレードの詳細記録用

    for i in range(steps):
        current_price = df["raw_close"].iloc[env.current_step]  # 実価格を取得

        # --- モデルにより行動を予測 ---
        action, _ = model.predict(obs)
        action = int(action)
        obs, reward, terminated, truncated, _ = env.step(action)

        # === エントリー処理 ===
        if action == 2 and position is None:
            # シンプルにステップの偶奇でポジション決定（ロング／ショート）
            position = "long" if i % 2 == 0 else "short"
            entry_price = current_price
            entry_step = i

        # === クローズ処理 ===
        elif action == 1 and position is not None:
            if position == "long":
                pnl = current_price - entry_price
            elif position == "short":
                pnl = entry_price - current_price
            else:
                pnl = 0

            # キャピタルに損益を反映
            capital += pnl

            # トレード記録を追加
            trade_log.append({
                "position": position,
                "entry_step": entry_step,
                "exit_step": i,
                "entry_price": entry_price,
                "exit_price": current_price,
                "pnl": pnl
            })
            position = None  # ポジションクローズ

        equity_curve.append(capital)  # 資産推移を記録

        # 終了条件
        if terminated or truncated:
            break

    # ========== 結果の集計・出力 ==========
    total_trades = len(trade_log)
    win_trades = sum(1 for t in trade_log if t["pnl"] > 0)
    total_pnl = sum(t["pnl"] for t in trade_log)
    max_drawdown = max(
        [max(equity_curve[:i+1]) - equity_curve[i] for i in range(len(equity_curve))]
    )

    # --- 評価レポート表示 ---
    print("📊 トレード損益ベース評価（ロング & ショート）")
    print(f"・評価期間: {df['time'].iloc[0]} ～ {df['time'].iloc[env.current_step]}")
    print(f"・総トレード数: {total_trades}")
    print(f"・勝率: {win_trades / total_trades * 100:.2f}%" if total_trades > 0 else "・勝率: 0%")
    print(f"・累積損益: {total_pnl:.2f}")
    print(f"・最終キャピタル: {capital:.2f}")
    print(f"・最大ドローダウン: {max_drawdown:.2f}")


# ========== 評価実行（Volatility AIモデルに対して） ==========
model = model  # 既にロード済みの学習モデル
eval_env = VolatilityEnv(df)  # VolatilityEnv 環境で df（raw_close 列が必要）を使用
evaluate_model_with_pnl(model, eval_env, steps=500)


📊 トレード損益ベース評価（ロング & ショート）
・評価期間: 2025-03-06 10:40:00 ～ 2025-03-10 05:20:00
・総トレード数: 13
・勝率: 61.54%
・累積損益: -0.58
・最終キャピタル: 9999.42
・最大ドローダウン: 11.17
